In [1]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob
np.random.seed(47)

Using TensorFlow backend.


In [2]:
def load_dataset(path):
    data = load_files(path)
    img_files = np.array(data['filenames'])
    img_target = np_utils.to_categorical(np.array(data['target']), num_classes=101)
    return img_files, img_target

In [3]:
train_files, train_target = load_dataset('101_ObjectCategories')

In [4]:
img_names = [item[21:] for item in sorted(glob('101_ObjectCategories/*'))]

In [5]:
print('There are %d total different categories.'%len(img_names))
print('There are %s total images in the dataset.'%len(train_files))

There are 101 total different categories.
There are 8677 total images in the dataset.


In [6]:
from keras.preprocessing import image

def path_to_tensor(img_path):
    #loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # Returns 4D tensor with shape (1, 224, 224, 3)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [7]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#### pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_tensors, train_target, test_size = 0.2, random_state=47)

print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

(6941, 224, 224, 3) (6941, 101)
(1736, 224, 224, 3) (1736, 101)


In [19]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential, Model

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 224, 224, 16)      208       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 224, 224, 16)      1040      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 112, 112, 32)      2080      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 112, 112, 32)      4128      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 56, 56, 32)        0         
__________

In [20]:
# compile the model
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [21]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='weights.best.hdf5', verbose=1, save_best_only=True)
hist = model.fit(x_train, y_train, batch_size=128, epochs=20,validation_data=(x_test, y_test), 
                 callbacks=[checkpointer], verbose=2, shuffle=True)

Train on 6941 samples, validate on 1736 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 4.27096, saving model to weights.best.hdf5
48s - loss: 4.1007 - acc: 0.1668 - val_loss: 4.2710 - val_acc: 0.2166
Epoch 2/20
Epoch 00001: val_loss improved from 4.27096 to 3.96832, saving model to weights.best.hdf5
49s - loss: 3.5279 - acc: 0.2625 - val_loss: 3.9683 - val_acc: 0.1653
Epoch 3/20
Epoch 00002: val_loss improved from 3.96832 to 3.24995, saving model to weights.best.hdf5
49s - loss: 3.2161 - acc: 0.3165 - val_loss: 3.2499 - val_acc: 0.3272
Epoch 4/20
Epoch 00003: val_loss improved from 3.24995 to 2.92763, saving model to weights.best.hdf5
49s - loss: 2.9606 - acc: 0.3533 - val_loss: 2.9276 - val_acc: 0.3882
Epoch 5/20
Epoch 00004: val_loss did not improve
45s - loss: 2.8163 - acc: 0.3772 - val_loss: 2.9507 - val_acc: 0.4009
Epoch 6/20
Epoch 00005: val_loss improved from 2.92763 to 2.73934, saving model to weights.best.hdf5
49s - loss: 2.5828 - acc: 0.4165 - val_loss: 2.7393 

In [22]:
model.load_weights('weights.best.hdf5')

In [24]:
# Score trained model
score = model.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.521313364055
